# Load Hugging Face Model

In [66]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.callbacks import StreamingStdOutCallbackHandler

repo_id = "google/gemma-2-2b-it"
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    streaming=True,
    temperature=0.7,
    callbacks=[StreamingStdOutCallbackHandler()]
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/cjung-mo/.cache/huggingface/token
Login successful


In [17]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [19]:
llm.invoke("What is Serialization and composition in prompt engineering?")



Let's break down these concepts in the context of prompt engineering for large language models (LLMs).

**1. Serialization**

* **What it is:** Serialization is the process of converting a data structure (like a Python list, dictionary, or a complex object) into a string format that can be easily stored, transmitted, or processed. Think of it as packaging your data into a neat, portable box.
* **Why it matters in prompt engineering:**  LLMs are powerful but can be picky about the format of the input they receive.  You might need to serialize your data into a specific format (like JSON) to ensure the LLM understands it correctly. 
* **Example:**  Let's say you want to provide a list of movie titles to an LLM for a movie recommendation task. You might serialize this list into JSON format:

```json
[
  "The Shawshank Redemption",
  "Pulp Fiction",
  "The Dark Knight"
]
```

**2. Composition**

* **What it is:** Composition is the process of combining multiple smaller components or eleme

'\n\nLet\'s break down these concepts in the context of prompt engineering for large language models (LLMs).\n\n**1. Serialization**\n\n* **What it is:** Serialization is the process of converting a data structure (like a Python list, dictionary, or a complex object) into a string format that can be easily stored, transmitted, or processed. Think of it as packaging your data into a neat, portable box.\n* **Why it matters in prompt engineering:**  LLMs are powerful but can be picky about the format of the input they receive.  You might need to serialize your data into a specific format (like JSON) to ensure the LLM understands it correctly. \n* **Example:**  Let\'s say you want to provide a list of movie titles to an LLM for a movie recommendation task. You might serialize this list into JSON format:\n\n```json\n[\n  "The Shawshank Redemption",\n  "Pulp Fiction",\n  "The Dark Knight"\n]\n```\n\n**2. Composition**\n\n* **What it is:** Composition is the process of combining multiple smal

# Serialization and Composition

## Serializatioin

In [11]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")
prompt.format(country="South Korea")

'what is the capital of South Korea'

## Composition

### [PipelinePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html#)

In [99]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    You are inpersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)

In [74]:
full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrg! That is a secret!",
    question="What is your favorite food?"
)

'\n    \n    You are a role playing assistant.\n    You are inpersonating a Pirate\n\n\n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrg! That is a secret!\n\n\n    \n    Start now!\n\n    Human: What is your favorite color?\n    You:\n\n'

In [77]:
chain = full_prompt | llm

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrg! That is a secret!",
    "question":"What is your favorite food?"
})

    
    Arrr! Me favorite food be a hearty stew with some good ol' dried meat. 
    



<end_of_turn>

"    \n    Arrr! Me favorite food be a hearty stew with some good ol' dried meat. \n    \n\n\n\n<end_of_turn>"

# [LLM Caching](https://python.langchain.com/docs/integrations/llm_caching/)

In [85]:
from langchain.globals import set_llm_cache, set_debug
from langchain_community.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
set_debug(True)

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/cjung-mo/.cache/huggingface/token
Login successful


## InMemoryCache

In [79]:
llm.predict("How to make Kimchi Stew?")

/tmp/ipykernel_43275/2688879220.py:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("How to make Kimchi Stew?")


[llm/start] [llm:HuggingFaceEndpoint] Entering LLM run with input:
{
  "prompts": [
    "How to make Kimchi Stew?"
  ]
}
[llm/end] [llm:HuggingFaceEndpoint] [8.61s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\nKimchi stew, also known as kimchi jjigae, is a popular Korean dish that is both comforting and flavorful. It's a hearty stew made with kimchi, tofu, pork, and other vegetables. \n\nHere's a basic recipe for kimchi stew:\n\n**Ingredients:**\n\n* 1 tbsp vegetable oil\n* 1/2 cup chopped onion\n* 1/2 cup chopped garlic\n* 1/2 cup chopped ginger\n* 1 lb pork belly, cut into 1-inch cubes\n* 1 tbsp gochujang (Korean chili paste)\n* 1/2 cup gochugaru (Korean chili flakes)\n* 1/4 cup soy sauce\n* 1/4 cup rice wine (optional)\n* 1 cup kimchi, chopped\n* 1/2 cup chopped tofu\n* 1/2 cup chopped mushrooms (optional)\n* 1/4 cup chopped green onions\n* 1 tbsp sesame oil\n* Salt and pepper to taste\n\n**Instructions:**\n\n1. Heat the vegetable oil in a lar

"\n\nKimchi stew, also known as kimchi jjigae, is a popular Korean dish that is both comforting and flavorful. It's a hearty stew made with kimchi, tofu, pork, and other vegetables. \n\nHere's a basic recipe for kimchi stew:\n\n**Ingredients:**\n\n* 1 tbsp vegetable oil\n* 1/2 cup chopped onion\n* 1/2 cup chopped garlic\n* 1/2 cup chopped ginger\n* 1 lb pork belly, cut into 1-inch cubes\n* 1 tbsp gochujang (Korean chili paste)\n* 1/2 cup gochugaru (Korean chili flakes)\n* 1/4 cup soy sauce\n* 1/4 cup rice wine (optional)\n* 1 cup kimchi, chopped\n* 1/2 cup chopped tofu\n* 1/2 cup chopped mushrooms (optional)\n* 1/4 cup chopped green onions\n* 1 tbsp sesame oil\n* Salt and pepper to taste\n\n**Instructions:**\n\n1. Heat the vegetable oil in a large pot or Dutch oven over medium heat.\n2. Add the onion, garlic, and ginger and cook until softened, about 5 minutes.\n3. Add the pork belly and cook until browned on all sides, about 5 minutes.\n4. Stir in the gochujang, gochugaru, soy sauce, 

In [80]:
llm.predict("How to make Kimchi Stew?")

"\n\nKimchi stew, also known as kimchi jjigae, is a popular Korean dish that is both comforting and flavorful. It's a hearty stew made with kimchi, tofu, pork, and other vegetables. \n\nHere's a basic recipe for kimchi stew:\n\n**Ingredients:**\n\n* 1 tbsp vegetable oil\n* 1/2 cup chopped onion\n* 1/2 cup chopped garlic\n* 1/2 cup chopped ginger\n* 1 lb pork belly, cut into 1-inch cubes\n* 1 tbsp gochujang (Korean chili paste)\n* 1/2 cup gochugaru (Korean chili flakes)\n* 1/4 cup soy sauce\n* 1/4 cup rice wine (optional)\n* 1 cup kimchi, chopped\n* 1/2 cup chopped tofu\n* 1/2 cup chopped mushrooms (optional)\n* 1/4 cup chopped green onions\n* 1 tbsp sesame oil\n* Salt and pepper to taste\n\n**Instructions:**\n\n1. Heat the vegetable oil in a large pot or Dutch oven over medium heat.\n2. Add the onion, garlic, and ginger and cook until softened, about 5 minutes.\n3. Add the pork belly and cook until browned on all sides, about 5 minutes.\n4. Stir in the gochujang, gochugaru, soy sauce, 

## SQLiteCache

In [83]:
set_llm_cache(SQLiteCache("cache.db"))

llm.predict("How to survive in a forest?")

[llm/start] [llm:HuggingFaceEndpoint] Entering LLM run with input:
{
  "prompts": [
    "How to survive in a forest?"
  ]
}
[llm/end] [llm:HuggingFaceEndpoint] [8.65s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\n**Disclaimer:** This information is for general knowledge and survival purposes only. It is not a substitute for professional training or emergency services.\n\n**1. Stay Calm and Assess the Situation:**\n\n* **Panic is your worst enemy.** Take deep breaths and assess your surroundings.\n* **Identify potential dangers:** Are there wild animals, poisonous plants, or unstable terrain?\n* **Check for injuries:** Treat any wounds immediately.\n\n**2. Shelter:**\n\n* **Find a natural shelter:** Look for a cave, overhang, or dense thicket.\n* **Build a lean-to:** Use branches, leaves, and debris to create a temporary shelter.\n* **Use natural materials:** Gather leaves, branches, and moss to insulate your shelter.\n\n**3. Water:**\n\n* **Find 

'\n\n**Disclaimer:** This information is for general knowledge and survival purposes only. It is not a substitute for professional training or emergency services.\n\n**1. Stay Calm and Assess the Situation:**\n\n* **Panic is your worst enemy.** Take deep breaths and assess your surroundings.\n* **Identify potential dangers:** Are there wild animals, poisonous plants, or unstable terrain?\n* **Check for injuries:** Treat any wounds immediately.\n\n**2. Shelter:**\n\n* **Find a natural shelter:** Look for a cave, overhang, or dense thicket.\n* **Build a lean-to:** Use branches, leaves, and debris to create a temporary shelter.\n* **Use natural materials:** Gather leaves, branches, and moss to insulate your shelter.\n\n**3. Water:**\n\n* **Find a water source:** Look for streams, rivers, or springs.\n* **Collect rainwater:** Use a tarp or container to collect rainwater.\n* **Purify water:** Boil water for at least one minute or use a water filter.\n\n**4. Fire:**\n\n* **Find tinder:** Loo

In [84]:
llm.predict("How to survive in a forest?")

'\n\n**Disclaimer:** This information is for general knowledge and survival purposes only. It is not a substitute for professional training or emergency services.\n\n**1. Stay Calm and Assess the Situation:**\n\n* **Panic is your worst enemy.** Take deep breaths and assess your surroundings.\n* **Identify potential dangers:** Are there wild animals, poisonous plants, or unstable terrain?\n* **Check for injuries:** Treat any wounds immediately.\n\n**2. Shelter:**\n\n* **Find a natural shelter:** Look for a cave, overhang, or dense thicket.\n* **Build a lean-to:** Use branches, leaves, and debris to create a temporary shelter.\n* **Use natural materials:** Gather leaves, branches, and moss to insulate your shelter.\n\n**3. Water:**\n\n* **Find a water source:** Look for streams, rivers, or springs.\n* **Collect rainwater:** Use a tarp or container to collect rainwater.\n* **Purify water:** Boil water for at least one minute or use a water filter.\n\n**4. Fire:**\n\n* **Find tinder:** Loo